In [17]:
import pandas as pd
import json
import re
import seaborn as sns
from plotly.subplots import make_subplots
import plotly.express as px
import time
import openai
import os
from dotenv import load_dotenv
import tiktoken

In [18]:
data = pd.read_csv("googlemaps_review.csv")
info = pd.read_csv("googlemaps_info.csv")

In [19]:
id_list = data["location_id"].unique().tolist()
id_list

['id_33', 'id_48', 'id_93']

### Extract the data from the reviews in json

In [20]:
rating_list = []
review_date_list = []
rating_cat_list = []
review_text_list = []
for id_num in id_list:
    
    current_df = data[data["location_id"]==id_num]
    current_review = current_df["moreReview"].tolist()
    
    for i in range(len(current_review)):
        # review in json object
        json_obj = json.loads(current_review[i])
        # extract ratings
        ratings = int(json_obj["rating"][0])
        rating_list.append(ratings)
        # categorize ratings
        if ratings >3:
            rating_cat_list.append("> 3 stars")
        else:
            rating_cat_list.append("<= 3 stars")
        # extract review date
        date = json_obj["review_date"]
        split_date = date.split(" ")[0:-1]
        if re.match("(year)s?",split_date[1]):
            review_date_list.append("> a year")
        else:
            review_date_list.append("<= a year")
        # extract review text content
        review_text_list.append(json_obj["review"])

data["review_rating"] = rating_list
data["review_rating_category"] = rating_cat_list
data["review_date"] = review_date_list
data["review_content"] = review_text_list

In [21]:
# Add the location name
data["location_name"] = data["location_id"].apply(lambda x: info[info["location_id"]==x]["location"].values[0])
data= data.reindex(columns = ["location_id", "location_name",
                              "moreReview", "review_rating","review_rating_category", "review_date", "review_content"])
data["location_name"] = data["location_name"].apply(lambda x : " ".join(x.split(" ")[1::]))

In [22]:
# Clean the location name 
data["location_name"] = data["location_name"].apply(lambda x: x[1:5] if not x.isascii() else x)
data["location_name"] = data["location_name"].apply(lambda x: x.replace("*", ""))

### Bar plot of the ratings based on group category

In [27]:
stats_rate_df = pd.DataFrame(data.groupby(['location_id','location_name', 'review_rating_category']).size().reset_index(name="count"))
stats_rate_df

,location_id,location_name,review_rating_category,count
0,id_33,香港大學,<= 3 stars,7
1,id_33,香港大學,> 3 stars,13
2,id_48,The Henry,<= 3 stars,9
3,id_48,The Henry,> 3 stars,11
4,id_93,Imperial Kennedy,<= 3 stars,8
5,id_93,Imperial Kennedy,> 3 stars,12


In [30]:
color_sequence = px.colors.sequential.RdBu

n_colors = 2
color_range = [i*(len(color_sequence)-1)//(n_colors-1) for i in range(n_colors)]
color_list = [color_sequence[idx] for idx in color_range]

fig = px.bar(stats_rate_df, 
             x="review_rating_category", y="count", 
             color="review_rating_category", color_discrete_sequence =color_list,
             text = "count",
             facet_col="location_name",
             facet_col_spacing =0.01,
             labels={"count":"Count", 
                     "review_rating_category":"Review Ratings",
                    "location_name": "Location"},
             title = "Bar Chart of the Ratings (Overview)"
            )

fig.update_traces(marker_line_color='black', marker_line_width=0.4)
fig.update_layout(showlegend=True,
                  legend=dict(title="Review Ratings:", orientation='h', yanchor='top',y=1.25,xanchor='center',x=0.5, bordercolor="#B0C4DE", borderwidth=1),
                  title=dict(y=0.95, x=0.5, xanchor='center', yanchor='top'), 
                  width=600, height=400
                 )

fig.write_image("1bar_overview.jpeg")
fig.show()


### Bar plot of the ratings based on ratings values

In [25]:
stats_df = pd.DataFrame(data.groupby(['location_id','location_name', 'review_rating', 'review_date']).size().reset_index(name="count"))
stats_df.review_rating = stats_df.review_rating.astype(str)
stats_df

,location_id,location_name,review_rating,review_date,count
0,id_33,香港大學,1,<= a year,2
1,id_33,香港大學,1,> a year,3
2,id_33,香港大學,3,> a year,2
3,id_33,香港大學,4,<= a year,1
4,id_33,香港大學,4,> a year,3
5,id_33,香港大學,5,> a year,9
6,id_48,The Henry,1,<= a year,2
7,id_48,The Henry,2,> a year,3
8,id_48,The Henry,3,> a year,4
9,id_48,The Henry,4,> a year,7


In [39]:
color_sequence = px.colors.sequential.RdBu

n_colors = 5
color_range = [i*(len(color_sequence)-1)//(n_colors-1) for i in range(n_colors)]
color_list = [color_sequence[idx] for idx in color_range]
custom_order= ["1","2","3","4","5"]
color_map = {key:val for (key, val) in zip(custom_order,color_list)}


fig = px.bar(stats_df, 
             x="review_rating", y="count", 
             color="review_rating", 
             color_discrete_map = color_map,
             text = "review_date",
             facet_col="location_name",
             facet_col_spacing =0.01,
             labels={"count":"Count", 
                     "review_rating":"Review Ratings",
                    "location_name": "Location"},
             title = "Bar Chart of the Ratings (Detail)"
            )

fig.update_traces(texttemplate='%{text}', 
                  textposition='inside', 
                  textfont=dict(size=10), 
                  textangle=45,
                  marker_line_color='black', marker_line_width=0.4)
fig.update_layout(showlegend=True,
                  xaxis={'categoryorder': 'array', 'categoryarray': custom_order},
                  legend=dict(title="Review Ratings:", orientation='h', yanchor='top',y=1.25,xanchor='center',x=0.5, bordercolor="#B0C4DE", borderwidth=1),
                  title=dict(y=0.95, x=0.5, xanchor='center', yanchor='top'), 
                  width=800, height=400
                 )
fig.write_image("2bar_detail.jpeg")
fig.show()

## chatgpt sentiment analysis

In [10]:
load_dotenv("config.env")
openai.api_key  = os.getenv('OPENAI_API_KEY')


In [11]:
def prompt_text(text):
    prompt= f""" Review is provided and delimited by triple backticks.\

    Determine the following information:
    1. What is the overall sentiment of the review?\
    Give your answer as either\
    'Positive', 'Neutral' or 'Negative'.\

    2. Determine the emotions that is portrayed in the review by the reviewer.\
    Give 3 emotions that is expressed by the reviewer. \
    Ensure each word is an adjective and do not repeat the word.\
    
    3. Determine if 'customer service' \
    was mentioned in the review.\
    Provide the answer in either 'Positive', 'Negative' or 'NA' only.\
    
    Format the output in dictionary format, and use\
    "Sentiment", \
    "Emotions",\
    "Customer Service" as the keys.\

    Review: '''{text}'''
    """
    return prompt

In [12]:
def model_response(prompt):
    response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages= [{"role": "user", "content": prompt}],
    temperature=0,
    )
    return response.choices[0].message["content"], response.usage.total_tokens

### chatgpt extract the data

In [14]:
output_list = []
total_tokens_list = []

for index in range(len(data)):
    
    if (index+1)%20 == 0:
        print("Starting the pause...")
        time.sleep(60)
        print("Pause is over.")
    
    print(f"\nReview No {index+1}\n")    
    text = data["review_content"].tolist()[index]

    prompt = prompt_text(text)
    output, total_tokens = model_response(prompt)
    
    output_refine = json.loads(output.replace("\n", ""))
    output_list.append(output_refine)
    
    total_tokens_list.append(total_tokens)  

data["gpt_output"] = output_list
data["gpt_total_tokens"] = total_tokens_list


Review No 1


Review No 2


Review No 3


Review No 4


Review No 5


Review No 6


Review No 7


Review No 8


Review No 9


Review No 10


Review No 11


Review No 12


Review No 13


Review No 14


Review No 15


Review No 16


Review No 17


Review No 18


Review No 19

Starting the pause...
Pause is over.

Review No 20


Review No 21


Review No 22


Review No 23


Review No 24


Review No 25


Review No 26


Review No 27


Review No 28


Review No 29


Review No 30


Review No 31


Review No 32


Review No 33


Review No 34


Review No 35


Review No 36


Review No 37


Review No 38


Review No 39

Starting the pause...
Pause is over.

Review No 40


Review No 41


Review No 42


Review No 43


Review No 44


Review No 45


Review No 46


Review No 47


Review No 48


Review No 49


Review No 50


Review No 51


Review No 52


Review No 53


Review No 54


Review No 55


Review No 56


Review No 57


Review No 58


Review No 59

Starting the pause...
Pause is over.

Review No 60

### process and create new columns for the extracted data

In [15]:
sentiment_list = []
service_list = []
emotions_list = []
for index in range(len(data)):
    gpt_output = data["gpt_output"].tolist()[index]
    sentiment_list.append(gpt_output["Sentiment"])
    service_list.append(gpt_output["Customer Service"])
    emotions_list.append(gpt_output["Emotions"])

data["gpt_output_sentiment"] = sentiment_list
data["gpt_output_service"] = service_list
data["gpt_output_emotions"] = emotions_list

In [16]:
data

,location_id,location_name,moreReview,review_rating,review_rating_category,review_date,review_content,gpt_output,gpt_total_tokens,gpt_output_sentiment,gpt_output_service,gpt_output_emotions
0,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""Coey"", ""rating"": ""1 star"", ""r...",1,<= 3 stars,<= a year,"the staff is insanely rude, the man with glass...","{'Sentiment': 'Negative', 'Emotions': ['Rude',...",337,Negative,Negative,"[Rude, Doubtful, Frustrated]"
1,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""Momo Yung"", ""rating"": ""1 star...",1,<= 3 stars,> a year,I am in a rush to get my car out of carpark bu...,"{'Sentiment': 'Negative', 'Emotions': ['Hungry...",297,Negative,Negative,"[Hungry, Disappointed, Ignored]"
2,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""TSZ LOK Cheng"", ""rating"": ""1 ...",1,<= 3 stars,<= a year,They don't accept cash to order coffee nor top...,"{'Sentiment': 'Negative', 'Emotions': ['Disapp...",227,Negative,NA,"[Disappointing, Frustrated, Unhappy]"
3,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""Jasper"", ""rating"": ""5 stars"",...",5,> 3 stars,> a year,30% discount for HKU students 😄 …,"{'Sentiment': 'Positive', 'Emotions': ['Happy'...",213,Positive,NA,"[Happy, Excited, Pleased]"
4,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""Patrick Bai"", ""rating"": ""5 st...",5,> 3 stars,> a year,"With the student discount, drinks are outrageo...","{'Sentiment': 'Positive', 'Emotions': ['Thrill...",228,Positive,NA,"[Thrilled, Pleased, Impressed]"
5,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""Jeshurun Ratnasamy"", ""rating""...",4,> 3 stars,<= a year,it’s a starbucks. not much to say apart from ...,"{'Sentiment': 'Neutral', 'Emotions': ['Conveni...",230,Neutral,NA,"[Convenient, Routine, Unremarkable]"
6,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""Tim Leung"", ""rating"": ""4 star...",4,> 3 stars,> a year,Reasonable price for HKU personnel. Only a sma...,"{'Sentiment': 'Neutral', 'Emotions': ['Reasona...",220,Neutral,NA,"[Reasonable, Small, Available]"
7,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""Terence Wan"", ""rating"": ""4 st...",4,> 3 stars,> a year,Cold Brew less ice,"{'Sentiment': 'Neutral', 'Emotions': ['Cold', ...",206,Neutral,NA,"[Cold, Less, Icy]"
8,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""Paul Fong"", ""rating"": ""1 star...",1,<= 3 stars,> a year,Staff not so professional n caring。。。。,"{'Sentiment': 'Negative', 'Emotions': ['Unprof...",213,Negative,NA,"[Unprofessional, Uncaring, Disappointing]"
9,id_33,香港大學綜合大樓地下2號舖,"{""author_name"": ""msblue737"", ""rating"": ""5 star...",5,> 3 stars,> a year,Good place to get a coffee in tai kok tsui.,"{'Sentiment': 'Positive', 'Emotions': ['Good',...",215,Positive,NA,"[Good, Enjoyable, Satisfying]"


### Histogram plot of review sentiment

In [17]:
custom_order = ["Negative", "Neutral", "Positive"]
color_list= ['rgb(103,0,31)','rgb(247,247,247)','rgb(5,48,97)']
color_map = {key:val for (key, val) in zip(custom_order,color_list)}


fig = px.histogram(data, x="gpt_output_sentiment", 
                   color="gpt_output_sentiment", 
                   color_discrete_map = color_map,

                   facet_col="location_name",
                   facet_col_spacing =0.01,
                   labels={"count":"Count", 
                     "gpt_output_sentiment":"Review Sentiment",
                    "location_name": "Location"},
                   title = "Histogram Chart of Review Sentiment"                  
                  )

fig.update_traces(marker_line_color='black', marker_line_width=0.4)
fig.update_layout(showlegend=True,
                  xaxis={'categoryorder': 'array', 'categoryarray': custom_order},
                  legend=dict(title="Review Ratings:", orientation='h', yanchor='top',y=1.15,xanchor='center',x=0.5, bordercolor="#B0C4DE", borderwidth=1),
                  title=dict(y=0.95, x=0.5, xanchor='center', yanchor='top'),
                 )
fig.write_image("1hist_overview.jpeg", width=800, height=500)
fig.show()

### Histogram plot of customer service standard

In [18]:
custom_order = ["Negative", "Positive", "NA"]
color_list= ['rgb(103,0,31)', 'rgb(5,48,97)', "grey"]
color_map = {key:val for (key, val) in zip(custom_order,color_list)}


fig = px.histogram(data, x="gpt_output_service", 
                   color="gpt_output_service", 
                   color_discrete_map = color_map,

                   facet_col="location_name",
                   facet_col_spacing =0.01,
                   labels={"count":"Count", 
                     "gpt_output_service":"Customer Service",
                    "location_name": "Location"},
                   title = "Histogram Chart of Customer Service Standard"                  
                  )

fig.update_traces(marker_line_color='black', marker_line_width=0.4)
fig.update_layout(showlegend=True,
                  xaxis={'categoryorder': 'array', 'categoryarray': custom_order},
                  legend=dict(title="Review Ratings:", orientation='h', yanchor='top',y=1.15,xanchor='center',x=0.5, bordercolor="#B0C4DE", borderwidth=1),
                  title=dict(y=0.95, x=0.5, xanchor='center', yanchor='top'),
                 )
fig.write_image("2hist_overview.jpeg", width=800, height=500)
fig.show()

### Save the data processed and extracted into csv file

In [19]:
data.to_csv("googlemaps_review_detail.csv", index=False)

### Convert the emotions data to be plotted on Tableau

In [20]:
emotion_list = []
location_list = []
for loc_id in data["location_name"].unique().tolist():
    # extract emotions one at a time by its location name
    current_data = data[data["location_name"]==loc_id]
    # combine all emotions into one list
    for i in current_data["gpt_output_emotions"]:
        # combine emotion words
        emotion_list += i
        # build a list based on location name
        location_list += [loc_id]*len(i)

# Save emotion data and location name to be used on Tableau
collect_df = pd.DataFrame(data = {"location_name":location_list ,"emotions":emotion_list})
collect_df.to_csv("googlemaps_tableau.csv", index=False)